In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import re

In [22]:
read_filename = Path(r"C:\U-Extend\IEEE_Codes\New_LABELS\26\10-18-22\026_10-18-22.txt")
save_filename = read_filename.with_suffix(".csv")

with open(read_filename, 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [23]:
lines

['file:///C:/U-Extend/IEEE_Codes/New_LABELS/26/10-18-22/026_10-18-22.eaf\n',
 'Monday, May 19, 2025, 4:09\u202fPM\n',
 '\n',
 'LEFT    KNOB_Reaching (Best)\n',
 'TC      00:00:24.255 - 00:00:25.240\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CW\n',
 'TC      00:00:26.260 - 00:00:26.913\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CW\n',
 'TC      00:00:26.952 - 00:00:27.252\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CW (Best)\n',
 'TC      00:00:27.709 - 00:00:28.142\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CCW (Best)\n',
 'TC      00:00:37.862 - 00:00:38.396\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CCW\n',
 'TC      00:00:39.170 - 00:00:39.537\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CCW\n',
 'TC      00:00:44.035 - 00:00:44.809\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CCW\n',
 'TC      00:00:45.486 - 00:00:46.400\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CCW\n',
 'TC      00:00:47.347 - 00:00:48.267\n',
 '\n',
 '\n',
 'LEFT    KNOB_Transport-CCW\n',
 'TC      00:00:53.783 - 00:

In [24]:
# Filter out empty or whitespace-only lines
filtered_lines = [line.strip() for line in lines if line.strip()]

# Process every two lines as a pair: (label line, TC line)
records = []
for i in range(2, len(filtered_lines), 2):  # skip the first two lines (file path and date)
    arm_label = filtered_lines[i]
    tc_line = filtered_lines[i + 1]
    
    arm, label = arm_label.split(None, 1)
    start, stop = tc_line.replace("TC", "").strip().split(" - ")
    
    records.append((label.strip(), arm.strip(), start.strip(), stop.strip()))

# Construct the DataFrame
df = pd.DataFrame(records, columns=["Label", "Arm", "Start", "Stop"])
df

,Label,Arm,Start,Stop
0,KNOB_Reaching (Best),LEFT,00:00:24.255,00:00:25.240
1,KNOB_Transport-CW,LEFT,00:00:26.260,00:00:26.913
2,KNOB_Transport-CW,LEFT,00:00:26.952,00:00:27.252
3,KNOB_Transport-CW (Best),LEFT,00:00:27.709,00:00:28.142
4,KNOB_Transport-CCW (Best),LEFT,00:00:37.862,00:00:38.396
5,KNOB_Transport-CCW,LEFT,00:00:39.170,00:00:39.537
6,KNOB_Transport-CCW,LEFT,00:00:44.035,00:00:44.809
7,KNOB_Transport-CCW,LEFT,00:00:45.486,00:00:46.400
8,KNOB_Transport-CCW,LEFT,00:00:47.347,00:00:48.267
9,KNOB_Transport-CCW,LEFT,00:00:53.783,00:00:54.546


In [25]:
df.to_csv(save_filename, index=False)

## For All Files

In [28]:
brooke_df = pd.read_csv('brooke_df.csv')
df1 = brooke_df.drop_duplicates(subset=['Subj#'], keep='first').reset_index(drop=True)
df1['Group'].value_counts()

Group
DMD        28
Healthy    23
SMA        12
Name: count, dtype: int64

In [29]:
label_dir = r"C:\U-Extend\IEEE_Codes\New_LABELS"

for index, row in brooke_df.iterrows():
    subj = str(row['Subj#']).strip()
    visit_date = str(row['Visit Date']).strip()
    visit_date_clean = pd.to_datetime(visit_date).strftime('%m-%d-%y')
    label_path = Path(os.path.join(label_dir, subj, visit_date_clean))

    txt_files = list(label_path.glob("*.txt"))

    if txt_files:
        read_filename = txt_files[0]
        save_filename = read_filename.with_suffix(".csv")

        print(read_filename)
        
        with open(read_filename, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Filter out empty or whitespace-only lines
        filtered_lines = [line.strip() for line in lines if line.strip()]
        # Process every two lines as a pair: (label line, TC line)
        records = []
        for i in range(2, len(filtered_lines), 2):  # skip the first two lines (file path and date)
            arm_label = filtered_lines[i]
            tc_line = filtered_lines[i + 1]
            
            arm, label = arm_label.split(None, 1)
            label = re.sub(r'_(?=Ext\b)', '-', label)
            label = re.sub(r'_(?=Flex\b)', '-', label)

            start, stop = tc_line.replace("TC", "").strip().split(" - ")
            records.append((label.strip(), arm.strip(), start.strip(), stop.strip()))
        
        # Construct the DataFrame
        df = pd.DataFrame(records, columns=["Label", "Arm", "Start", "Stop"])
        df.to_csv(save_filename, index=False)

C:\U-Extend\IEEE_Codes\New_LABELS\1\01-27-21\01_03-26-21.txt
C:\U-Extend\IEEE_Codes\New_LABELS\1\07-08-21\001_07-08-21.txt
C:\U-Extend\IEEE_Codes\New_LABELS\1\11-10-21\01_11-10-21.txt
C:\U-Extend\IEEE_Codes\New_LABELS\1\08-15-23\001_08-15-23.txt
C:\U-Extend\IEEE_Codes\New_LABELS\1\02-20-24\01-2-20-24.txt
C:\U-Extend\IEEE_Codes\New_LABELS\1\08-28-24\01-8-28-24.txt
C:\U-Extend\IEEE_Codes\New_LABELS\2\02-11-21\002_2-11-21_P2P.txt
C:\U-Extend\IEEE_Codes\New_LABELS\3\02-24-21\003_2-24-21.txt
C:\U-Extend\IEEE_Codes\New_LABELS\3\07-20-21\003_7-20-21.txt
C:\U-Extend\IEEE_Codes\New_LABELS\3\01-26-22\003_1-26-22.txt
C:\U-Extend\IEEE_Codes\New_LABELS\5\03-11-21\005_3-11-21_P2P.txt
C:\U-Extend\IEEE_Codes\New_LABELS\7\04-21-21\007_4-21-21_P2P.txt
C:\U-Extend\IEEE_Codes\New_LABELS\8\04-21-21\008_4-21-21.txt
C:\U-Extend\IEEE_Codes\New_LABELS\9\03-24-21\009_03-24-21.txt
C:\U-Extend\IEEE_Codes\New_LABELS\9\09-22-21\009_09-22-21.txt
C:\U-Extend\IEEE_Codes\New_LABELS\9\03-23-22\009_3-23-22.txt
C:\U-Exten